In [1]:
from os import path, listdir
import pandas as pd
from tqdm.notebook import tqdm
from os import listdir, path, stat
import logging
from xbrl.cache import HttpCache
from xbrl.instance import XbrlParser
import json
import uuid
from pymongo import MongoClient
from datetime import datetime


In [2]:
from digiaccounts.digiaccounts_data import (
    get_financial_table,
    get_startend_period,
    get_company_address,
    get_company_registration,
    get_accounting_software,
    get_share_info,
    get_director_names,
    get_company_postcodes,
    get_average_employees,
    get_dormant_state
)

from digiaccounts.digiaccounts_io import (
    get_account_information_dictionary,
    add_account_to_collection,
    create_unique_id
)

In [3]:
def get_database(dbname):

    CONNECTION_STRING = 'mongodb://localhost:27017'

    client = MongoClient(CONNECTION_STRING)

    return client[dbname]

In [4]:
mongodb = get_database('annual_accounts_db')
filed_accounts = mongodb['filed_accounts']
index_accounts = mongodb['index_accounts']

In [48]:
documents = filed_accounts.distinct('registration_number')
len(documents)

946

In [49]:
documents = filed_accounts.aggregate(
    [ 
        {   
            "$group":  { 
                    "_id": "$registration_number", 
                    "count": { "$sum": 1 } 
                } 
        },
        {
            "$group": {
                "_id": None, 
                "maxCount": { "$max": "$count" },
                "minCount": { "$min": "$count" },
                "avgCount": { "$avg": "$count" }
            }
        }
    ]    
)
for d in documents:
    print(d)

{'_id': None, 'maxCount': 4, 'minCount': 1, 'avgCount': 3.799154334038055}


In [47]:
documents = filed_accounts.aggregate(
    [
        {
            "$match": {
                "$and": [
                    {"$expr": {"$lt": [{"$dateFromString": {"dateString": "$period_ending"}}, datetime.strptime('20180331', "%Y%m%d")]}},
                    {"is_dormant": {'$eq':False}},
                    {"$expr": {"$gt": [{"$toDouble": "$average_employees"}, 0]}}
                ]
            }
        },
        {
           "$group": {
                "_id": 'Ending 2018', 
                "entityCount": {"$sum": 1},
                "maxEmployees": { "$max": {"$toDouble": "$average_employees" } },
                "minEmployees": { "$min": {"$toDouble": "$average_employees" } },
                "avgEmployees": { "$avg": {"$toDouble": "$average_employees" } }
            } 
        }
    ]
)
for d in documents:
    print(d)

documents = filed_accounts.aggregate(
    [
        {
            "$match": {
                "$and": [
                    {"$expr": {"$gte": [{"$dateFromString": {"dateString": "$period_ending"}}, datetime.strptime('20180331', "%Y%m%d")]}},
                    {"$expr": {"$lt": [{"$dateFromString": {"dateString": "$period_ending"}}, datetime.strptime('20190331', "%Y%m%d")]}},
                    {"is_dormant": {'$eq':False}},
                    {"$expr": {"$gt": [{"$toDouble": "$average_employees"}, 0]}}
                ]
                
            }
        },
        {
           "$group": {
                "_id": 'Ending 2019', 
                "entityCount": {"$sum": 1},
                "maxEmployees": { "$max": {"$toDouble": "$average_employees" } },
                "minEmployees": { "$min": {"$toDouble": "$average_employees" } },
                "avgEmployees": { "$avg": {"$toDouble": "$average_employees" } }
            } 
        }
    ]
)
for d in documents:
    print(d)

documents = filed_accounts.aggregate(
    [
        {
            "$match": {
                    "$and": [
                    {"$expr": {"$gte": [{"$dateFromString": {"dateString": "$period_ending"}}, datetime.strptime('20190331', "%Y%m%d")]}},
                    {"$expr": {"$lt": [{"$dateFromString": {"dateString": "$period_ending"}}, datetime.strptime('20200331', "%Y%m%d")]}},
                    {"is_dormant": {'$eq':False}},
                    {"$expr": {"$gt": [{"$toDouble": "$average_employees"}, 0]}}
                ]
            }
        },
        {
           "$group": {
                "_id": 'Ending 2020',
                "entityCount": {"$sum": 1}, 
                "maxEmployees": { "$max": {"$toDouble": "$average_employees" } },
                "minEmployees": { "$min": {"$toDouble": "$average_employees" } },
                "avgEmployees": { "$avg": {"$toDouble": "$average_employees" } }
            } 
        }
    ]
)
for d in documents:
    print(d)

documents = filed_accounts.aggregate(
    [
        {
            "$match": {
                "$and": [
                    {"$expr": {"$gte": [{"$dateFromString": {"dateString": "$period_ending"}}, datetime.strptime('20200331', "%Y%m%d")]}},
                    {"$expr": {"$lt": [{"$dateFromString": {"dateString": "$period_ending"}}, datetime.strptime('20210331', "%Y%m%d")]}},
                    {"is_dormant": {'$eq':False}},
                    {"$expr": {"$gte": [{"$toDouble": "$average_employees"}, 1]}}
                ]
            }
        },
        {
            "$group": {
                "_id": 'Ending 2021',
                "entityCount": {"$sum": 1},
                "maxEmployees": {"$max": {"$toDouble": "$average_employees"}},
                "minEmployees": {"$min": {"$toDouble": "$average_employees"}},
                "avgEmployees": {"$avg": {"$toDouble": "$average_employees"}}
            }
        }
    ]
)
for d in documents:
    print(d)


{'_id': 'Ending 2018', 'entityCount': 193, 'maxEmployees': 71.0, 'minEmployees': 1.0, 'avgEmployees': 5.44559585492228}
{'_id': 'Ending 2019', 'entityCount': 220, 'maxEmployees': 73.0, 'minEmployees': 1.0, 'avgEmployees': 5.345454545454546}
{'_id': 'Ending 2020', 'entityCount': 290, 'maxEmployees': 133.0, 'minEmployees': 1.0, 'avgEmployees': 5.0344827586206895}
{'_id': 'Ending 2021', 'entityCount': 502, 'maxEmployees': 148.0, 'minEmployees': 1.0, 'avgEmployees': 4.075697211155378}


In [50]:
query1 = {'$and':[
    
        {"$expr": {
            "$lt": [{ "$dateFromString": { "dateString": "$period_ending" }}, datetime.strptime('20180331', "%Y%m%d") ]
        }},
        {"is_dormant": {'$eq':True}}
    ]
}

query2 = {'$and':[
    
        {"$expr": {
            "$gte": [{ "$dateFromString": { "dateString": "$period_ending" }}, datetime.strptime('20200331', "%Y%m%d") ]
        }},
        {"is_dormant": {'$eq':False}}
    ]
}

query_comb = {'$or': [
        query1,
        query2
    ]
}

documents = filed_accounts.aggregate(
    [
        {
            "$match": query_comb
        },
        {
            "$group": {
                "_id": "$registration_number",
                "reg_filter_count": {"$sum": 1} 
            }
        },
        {
            "$match": {
                "$expr": {
                    "$gt": ["$reg_filter_count", 1]
                }
            }
        }
    ]
)
print('Started/Resumed Trading')
for d in documents:
    print(d['_id'])

Started/Resumed Trading
10580630
04515463
10247582
08121495
03711522
09657764
10256916
10628870
07277420
06578711


In [53]:
query1 = {'$and':[
    
        {"$expr": {
            "$lt": [{ "$dateFromString": { "dateString": "$period_ending" }}, datetime.strptime('20180331', "%Y%m%d") ]
        }},
        {"is_dormant": {'$eq':False}}
    ]
}

query2 = {'$and':[
    
        {"$expr": {
            "$gt": [{ "$dateFromString": { "dateString": "$period_ending" }}, datetime.strptime('20200331', "%Y%m%d") ]
        }},
        {"is_dormant": {'$eq':True}}
    ]
}

query_comb = {'$or': [
        query1,
        query2
    ]
}

documents = filed_accounts.aggregate(
    [
        {
            "$match": query_comb
        },
        {
            "$group": {
                "_id": "$registration_number",
                "reg_filter_count": {"$sum": 1} 
            }
        },
        {
            "$match": {
                "$expr": {
                    "$gt": ["$reg_filter_count", 1]
                }
            }
        }
    ]
)
print('Became Dormant')
for d in documents:
    print(d['_id'])

Became Dormant
05846658
02486387
06581433
07683147
06611142
06960777
08103435
07039319
08422579
09089125
05720886
05147948
05236303
00681513
07859429
06014577
09426738
06528091
08582970
09624376
3002097
03143441
10604085
10209911


In [52]:
aggrigation = filed_accounts.aggregate([
    {"$match": query_comb},
    {"$group": {"_id": "$registration_number", }}
])
results = [a for a in aggrigation]

In [53]:
results

[{'_id': '04515463'},
 {'_id': '02724445'},
 {'_id': '10249069'},
 {'_id': '06822086'},
 {'_id': '04788098'},
 {'_id': '06945910'},
 {'_id': '06785422'},
 {'_id': '03803105'},
 {'_id': '09466064'},
 {'_id': '09093655'},
 {'_id': '08120951'},
 {'_id': '09630826'},
 {'_id': '07660429'},
 {'_id': '04714517'},
 {'_id': '05837583'},
 {'_id': '09990722'},
 {'_id': 'SC220625'},
 {'_id': '08111277'},
 {'_id': 'SC380916'},
 {'_id': '10287329'},
 {'_id': '08553008'},
 {'_id': '06943351'},
 {'_id': '05851896'},
 {'_id': '10630084'},
 {'_id': '02619496'},
 {'_id': '10596497'},
 {'_id': 'SC344529'},
 {'_id': 'SC263539'},
 {'_id': '06937352'},
 {'_id': '04790107'},
 {'_id': '05141301'},
 {'_id': '06270022'},
 {'_id': '05990787'},
 {'_id': '01629495'},
 {'_id': '02244066'},
 {'_id': '02648944'},
 {'_id': '07546122'},
 {'_id': '05473296'},
 {'_id': '06295115'},
 {'_id': '06972653'},
 {'_id': '02213854'},
 {'_id': '10223920'},
 {'_id': '05492983'},
 {'_id': '05142168'},
 {'_id': '06831118'},
 {'_id': '

In [28]:
root = path.join('C:/Users', 'ahoward', 'Downloads')
dir_2018 = 'Accounts_Monthly_Data-March2018'
dir_2019 = 'Accounts_Monthly_Data-March2019'
dir_2020 = 'Accounts_Monthly_Data-March2020'
dir_2021 = 'Accounts_Monthly_Data-March2021'

accounts_2018 = path.join(root, dir_2018)
accounts_2019 = path.join(root, dir_2019)
accounts_2020 = path.join(root, dir_2020)
accounts_2021 = path.join(root, dir_2021)

In [29]:
files_2018 = listdir(accounts_2018)
files_2019 = listdir(accounts_2019)
files_2020 = listdir(accounts_2020)
files_2021 = listdir(accounts_2021)

In [30]:
files = [path.join(accounts_2018, f) for f in files_2018]
files += [path.join(accounts_2019, f) for f in files_2019]
files += [path.join(accounts_2020, f) for f in files_2020]
files += [path.join(accounts_2021, f) for f in files_2021]

# all_size = 0
# for f in tqdm(files):
#     all_size += stat(f).st_size
# print(f'All files size: {all_size} B')
# print(f'All files size: {all_size / (1024**2)} MB')
# print(f'All files size: {all_size / (1024**3)} GB')
len(files)

945782

In [31]:
df18 = pd.DataFrame(data=files_2018, columns = ['Files'])
df18['RegNo'] = df18['Files'].str.split('_', expand=True)[2]
df18['Date'] = pd.to_datetime(df18['Files'].str.split('_', expand=True)[3].str.split('.', expand=True)[0], format='%Y%m%d')
df18['Files'] = dir_2018 + '/' + df18['Files'].astype(str)

df19 = pd.DataFrame(data=files_2019, columns = ['Files'])
df19['RegNo'] = df19['Files'].str.split('_', expand=True)[2]
df19['Date'] = pd.to_datetime(df19['Files'].str.split('_', expand=True)[3].str.split('.', expand=True)[0], format='%Y%m%d')
df19['Files'] = dir_2019 + '/' + df19['Files'].astype(str)

df20 = pd.DataFrame(data=files_2020, columns = ['Files'])
df20['RegNo'] = df20['Files'].str.split('_', expand=True)[2]
df20['Date'] = pd.to_datetime(df20['Files'].str.split('_', expand=True)[3].str.split('.', expand=True)[0], format='%Y%m%d')
df20['Files'] = dir_2020 + '/' + df20['Files'].astype(str)

df21 = pd.DataFrame(data=files_2021, columns = ['Files'])
df21['RegNo'] = df21['Files'].str.split('_', expand=True)[2]
df21['Date'] = pd.to_datetime(df21['Files'].str.split('_', expand=True)[3].str.split('.', expand=True)[0], format='%Y%m%d')
df21['Files'] = dir_2021 + '/' + df21['Files'].astype(str)


In [32]:
print(f'18: {len(df18)}')
print(f'19: {len(df19)}')
10753 * 4

18: 195484
19: 212415


43012

In [33]:
df_all = (
    df18
    .merge(df19, 'inner', 'RegNo', suffixes=('_18', '_19'))
    .merge(df20, 'inner', 'RegNo', suffixes=('_19', '_20'))
    .merge(df21, 'inner', 'RegNo', suffixes=('_20', '_21'))
)

In [34]:
df_all.shape

(10753, 9)

In [54]:
df_samp = df_all.sample(100, random_state=100)
len(df_samp['Files_18'].to_list())

100

In [55]:
years = [18, 19, 20, 21]
file_list = []
for y in tqdm(years):
    file_list += df_samp[f'Files_{y}'].tolist()

  0%|          | 0/4 [00:00<?, ?it/s]

In [56]:
total_size = 0
for f in file_list:
    total_size += stat(path.join(root, f)).st_size

print(f'Combined file size: {total_size} B')
print(f'Combined file size: {total_size / (1024**2)} MB')

Combined file size: 32339073 B
Combined file size: 30.840943336486816 MB


In [57]:
logging.basicConfig(level=logging.ERROR)

cache = HttpCache('./cache')
cache.set_headers({'From': 'ahoward@companieshouse.gov.uk', 'User-Agent': 'py-xbrl/2.1.0'})
parser = XbrlParser(cache)

reported_dict = {}
lookup = None


errors = 0
no_given_address = 0
problem_instances = []
problem_addresses = []

for f in (pbar := tqdm(file_list)):
    pbar.set_description(f'Parse Errors: {errors}')
    account_path = path.join(root, f)
    
    parsed = False
    try:
        xbrl_instance = parser.parse_instance(account_path)
        parsed = True
    except:
        problem_instances.append(f)
        errors += 1
    

    if parsed:
        unique_id = create_unique_id(f)
        account_dictionary = get_account_information_dictionary(unique_id, xbrl_instance)
        add_account_to_collection(filed_accounts, account_dictionary)


with open('problem_files.json', 'w') as f:
    json.dump(problem_instances, f)


  0%|          | 0/400 [00:00<?, ?it/s]

In [65]:
f = problem_instances[1]
accounts_errors = []
for f in problem_instances:
    #print(f)
    account_path = path.join(root, f)
    try:
        xbrl_instance = parser.parse_instance(account_path)
    except KeyError as e:
        accounts_errors.append([str(e), account_path])
    except Exception as e:
        accounts_errors.append([str(e), account_path])
with open('account_errors.json', 'w') as o:
    json.dump(accounts_errors, o)

In [66]:
accounts_errors

[["'dpl-countries'",
  'C:/Users\\ahoward\\Downloads\\Accounts_Monthly_Data-March2018/Prod224_0052_07668935_20170630.html'],
 ["'dpl-countries'",
  'C:/Users\\ahoward\\Downloads\\Accounts_Monthly_Data-March2018/Prod224_0052_05697829_20180228.html'],
 ["'dpl-countries'",
  'C:/Users\\ahoward\\Downloads\\Accounts_Monthly_Data-March2018/Prod224_0052_03801700_20170630.html'],
 ["'charmap' codec can't decode byte 0x9d in position 28349: character maps to <undefined>",
  'C:/Users\\ahoward\\Downloads\\Accounts_Monthly_Data-March2018/Prod224_0052_04430950_20170630.html'],
 ["'dpl-countries'",
  'C:/Users\\ahoward\\Downloads\\Accounts_Monthly_Data-March2018/Prod224_0052_SC344948_20170630.html'],
 ["'dpl-countries'",
  'C:/Users\\ahoward\\Downloads\\Accounts_Monthly_Data-March2018/Prod224_0052_08584936_20170630.html'],
 ["'charmap' codec can't decode byte 0x9d in position 32810: character maps to <undefined>",
  'C:/Users\\ahoward\\Downloads\\Accounts_Monthly_Data-March2018/Prod224_0052_0730832